In [2]:
from tinygrad.runtime.ops_llvm import LLVMDevice, LLVMProgram, LLVMCompiler
import numpy as np
from llvmlite import ir
from tinygrad.device import MallocAllocator
from tinygrad.helpers import flat_mv
from functools import partialmethod
from tinygrad.dtype import dtypes
from tinygrad.renderer.llvmir import const

In [3]:
class AMX:
  @staticmethod
  def nop_op_imm5(op, imm5, builder): builder.asm(ir.FunctionType(ir.VoidType(), []), f".word (0x201000 + ({op} << 5) + {imm5}); amx op {op} imm {imm5}", "", tuple(), True)
  @staticmethod
  def op_gpr(op, builder, gpr): builder.asm(ir.FunctionType(ir.VoidType(), [ir.IntType(64)]), f".word (0x201000 + ({op} << 5) + 0$0 - ((0$0 >> 4) * 6)); amx op {op} reg $0", "r", (gpr,), True)
  set, clr = partialmethod(nop_op_imm5, 17, 0), partialmethod(nop_op_imm5, 17, 1)
  ldx, ldy, stx, sty = partialmethod(op_gpr, 0), partialmethod(op_gpr, 1), partialmethod(op_gpr, 2), partialmethod(op_gpr, 3)
  ldz, stz, ldzi, stzi = partialmethod(op_gpr, 4), partialmethod(op_gpr, 5), partialmethod(op_gpr, 6), partialmethod(op_gpr, 7)
  extrx, extry = partialmethod(op_gpr, 8), partialmethod(op_gpr, 9)
  fma64, fms64, fma32, fms32 = partialmethod(op_gpr, 10), partialmethod(op_gpr, 11), partialmethod(op_gpr, 12), partialmethod(op_gpr, 13)
  mac16, fma16, fms16 = partialmethod(op_gpr, 14), partialmethod(op_gpr, 15), partialmethod(op_gpr, 16)
  vecint, vecfp, matint, matfp, genlut = partialmethod(op_gpr, 18), partialmethod(op_gpr, 19), partialmethod(op_gpr, 20), partialmethod(op_gpr, 21), partialmethod(op_gpr, 22)

In [4]:
N = 32
na = np.zeros(N, dtype=np.float32)
# na = np.zeros((N, N), dtype=np.float32)
nb = np.random.randn(N, N).astype(np.float32)
nc = np.random.randn(N, N).astype(np.float32)

a = MallocAllocator.alloc(na.size * np.dtype(np.float32).itemsize)
b = MallocAllocator.alloc(nb.size * np.dtype(np.float32).itemsize)
c = MallocAllocator.alloc(nc.size * np.dtype(np.float32).itemsize)

MallocAllocator.copyin(b, flat_mv(nb.data))
MallocAllocator.copyin(c, flat_mv(nc.data))

In [5]:
module = ir.Module(name="amx_test")
func = ir.Function(module, ir.FunctionType(ir.IntType(64), [ir.FloatType().as_pointer()]*3), "amx")

entry = ir.IRBuilder(func.append_basic_block(name="entry"))
zm, xm, ym = [entry.ptrtoint(func.args[i], ir.IntType(64)) for i in range(3)]
exit = ir.IRBuilder(func.append_basic_block(name="exit"))

entry.branch(exit._block)

<ir.Branch '.8' of type 'void', opname 'br', operands [<ir.Block 'exit' of type 'label'>]>

In [6]:
AMX.fma32(exit, const(1 << 63 | 1 << 28, dtypes.int64))

exit.ret(const(0, dtypes.int64))

<ir.Ret '.10' of type 'void', opname 'ret', operands [<ir.Constant type='i64' value=0>]>

In [7]:
device = LLVMDevice("llvm")
ir_str = str(module)
print(ir_str)
prog = LLVMProgram(device, "amx", LLVMCompiler(device).compile(ir_str))

; ModuleID = "amx_test"
target triple = "unknown-unknown-unknown"
target datalayout = ""

define i64 @"amx"(float* %".1", float* %".2", float* %".3")
{
entry:
  %".5" = ptrtoint float* %".1" to i64
  %".6" = ptrtoint float* %".2" to i64
  %".7" = ptrtoint float* %".3" to i64
  br label %"exit"
exit:
  call void asm sideeffect ".word (0x201000 + (12 << 5) + 0$0 - ((0$0 >> 4) * 6)); amx op 12 reg $0", "r"
(i64 9223372037123211264)
  ret i64 0
}



In [1]:
import llvmlite.binding as llvm
mod = llvm.parse_assembly(ir_str)
print(device.target_machine.emit_assembly(mod))

NameError: name 'ir_str' is not defined

In [ ]:
prog(a, b, c, N**2)
MallocAllocator.copyout(flat_mv(na.data), a)

print(na)

: 